In [ ]:
import sys
import time
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config
from context import Context
from tools.runners.account_state_runner import get_account_keys_online
from tools.runners.account_state_runner import report_key_files_compare
from contracts.contract_identities import DEXContractInterface

context = Context()

In [ ]:
import time

# wrap the upgrade function of each contract around a state compare procedure
def upgrade_contract_with_check(contract: DEXContractInterface, *args):
    try:
        # get the current state of the account
        get_account_keys_online(contract.address, config.DEFAULT_PROXY, with_save_in="state_dump/pre_contract_state.json")
    except Exception as e:
        print(f"Error getting the account state: {e}")
        # ask for user accept to continue or cancel
        if input("Continue without state? (y/n): ") != "y":
            return

    # upgrade the contract
    contract.contract_upgrade(*args)

    time.sleep(12)

    try:
        # get the new state of the account
        get_account_keys_online(contract.address, config.DEFAULT_PROXY, with_save_in="state_dump/post_contract_state.json")
        # compare the states
        report_key_files_compare("state_dump", "pre", "post", True)
    except Exception as e:
        print(f"Error getting the account state: {e}")

Energy contract setup

In [ ]:
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
energy_contract: SimpleLockEnergyContract
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

In [ ]:
energy_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.SIMPLE_LOCK_ENERGY_BYTECODE_PATH, [], True)

In [ ]:
upgrade_contract_with_check(energy_contract, context.deployer_account, context.network_provider.proxy, config.SIMPLE_LOCK_ENERGY_BYTECODE_PATH, [], True)

Proxy dex setup

In [ ]:
from contracts.dex_proxy_contract import DexProxyContract
from utils.utils_chain import WrapperAddress as Address, hex_to_string
from utils.contract_data_fetchers import ProxyContractDataFetcher

dex_proxy_contract: DexProxyContract
dex_proxy_contract = context.get_contracts(config.PROXIES_V2)[0]

data_fetcher = ProxyContractDataFetcher(Address(dex_proxy_contract.address), context.network_provider.proxy.url)
old_locked_token = hex_to_string(data_fetcher.get_data("getLockedTokenIds")[0])
old_factory_address = context.get_contracts(config.LOCKED_ASSETS)[0].address
print(f"old_locked_token: {old_locked_token}")
print(f"old_factory_address: {old_factory_address}")

In [ ]:
dex_proxy_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.PROXY_V2_BYTECODE_PATH, [old_locked_token, old_factory_address])

In [ ]:
upgrade_contract_with_check(dex_proxy_contract, context.deployer_account, context.network_provider.proxy, config.PROXY_V2_BYTECODE_PATH, [old_locked_token, old_factory_address])

Farm contract setup

In [ ]:
from contracts.farm_contract import FarmContract
farm_contract: FarmContract
farm_contracts = context.get_contracts(config.FARMS_V2)

In [ ]:
from contracts.farm_contract import FarmContract
from tools.runners.farm_runner import get_farm_addresses_from_chain
from utils.contract_retrievers import retrieve_farm_by_address
farm_contract: FarmContract
farm_contracts: list[FarmContract] = []
farm_addresses = get_farm_addresses_from_chain("v2")
for farm_address in farm_addresses:
    farm_contract = retrieve_farm_by_address(farm_address)
    farm_contracts.append(farm_contract)
print(f"Loaded up {len(farm_contracts)} contracts")

In [ ]:
from utils.utils_chain import WrapperAddress, base64_to_hex
for farm_contract in farm_contracts:
    codehash = base64_to_hex(context.network_provider.proxy.get_account(WrapperAddress(farm_contract.address)).code_hash)
    if codehash == "ac3123e4c3bfe2a0947329a08f0a6b9899bc00b461f952f1c34974059689c6dc":
        print(f"Skipping {farm_contract.address}")
        continue
    farm_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.FARM_V3_BYTECODE_PATH, [], True)

In [ ]:
for farm_contract in farm_contracts:
    upgrade_contract_with_check(farm_contract, context.deployer_account, context.network_provider.proxy, config.FARM_V3_BYTECODE_PATH, [], True)

In [ ]:
# resume all
context.deployer_account.sync_nonce(context.network_provider.proxy)

for contract in farm_contracts:
    tx_hash = contract.resume(context.deployer_account, context.network_provider.proxy)

Staking contract setup

In [ ]:
from contracts.staking_contract import StakingContract
staking_contract: StakingContract
staking_contracts = context.get_contracts(config.STAKINGS_V2)

In [ ]:
from contracts.staking_contract import StakingContract, StakingContractVersion
from tools.runners.staking_runner import get_staking_addresses_from_chain
from utils.contract_retrievers import retrieve_staking_by_address
staking_contract: StakingContract
staking_contracts: list[StakingContract] = []
staking_addresses = get_staking_addresses_from_chain()
for staking_address in staking_addresses:
    staking_contract = retrieve_staking_by_address(staking_address, StakingContractVersion.V2)
    staking_contracts.append(staking_contract)
print(f"Loaded up {len(staking_contracts)} contracts")

In [ ]:
from utils.utils_chain import Account, WrapperAddress

# fix ownership
for contract in staking_contracts:
    owner = context.network_provider.proxy.get_account(WrapperAddress(contract.address)).owner_address
    if owner.to_bech32() != context.deployer_account.address.to_bech32():
        print(f"Found {contract.address} with owner {owner.to_bech32()}")
        owner_account = Account(pem_file=config.DEFAULT_ADMIN)
        owner_account.address = owner
        owner_account.sync_nonce(context.network_provider.proxy)

        contract.change_owner_address(owner_account, context.network_provider.proxy, context.deployer_account.address.bech32())
        contract.update_owner_or_admin(context.deployer_account, context.network_provider.proxy, owner.to_bech32())
        time.sleep(7)

In [ ]:
from utils.utils_chain import WrapperAddress, base64_to_hex, Account
for staking_contract in staking_contracts:
    codehash = base64_to_hex(context.network_provider.proxy.get_account(WrapperAddress(staking_contract.address)).code_hash)
    if codehash == "9ac73417f256997d656dbf92ff9d1b5d5d530b9e9d2b2315c239d65b8b9e9ab3":
        print(f"Skipping {staking_contract.address}")
        continue
    staking_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.STAKING_V3_BYTECODE_PATH, [], True)

In [ ]:
for staking_contract in staking_contracts:
    upgrade_contract_with_check(staking_contract, context.deployer_account, context.network_provider.proxy, config.STAKING_V3_BYTECODE_PATH, [], True)

In [ ]:
from time import sleep
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
context.deployer_account.sync_nonce(context.network_provider.proxy)
simple_lock_energy_contract: SimpleLockEnergyContract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

for contract in staking_contracts:
    tx_hash = contract.set_boosted_yields_rewards_percentage(context.deployer_account, context.network_provider.proxy, 6000)
    tx_hash = contract.set_boosted_yields_factors(context.deployer_account, context.network_provider.proxy, 
                                                  [2, 1, 0, 1, 1])
    tx_hash = contract.set_energy_factory_address(context.deployer_account, context.network_provider.proxy, context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0].address)
    tx_hash = simple_lock_energy_contract.add_sc_to_whitelist(context.deployer_account, context.network_provider.proxy, contract.address)
    sleep(2)

In [ ]:
# resume all
context.deployer_account.sync_nonce(context.network_provider.proxy)

for contract in staking_contracts:
    tx_hash = contract.resume(context.deployer_account, context.network_provider.proxy)

Metastaking contract setup

In [ ]:
from contracts.metastaking_contract import MetaStakingContract
metastaking_contract: MetaStakingContract
metastaking_contracts = context.get_contracts(config.METASTAKINGS_V2)

In [ ]:
from contracts.metastaking_contract import MetaStakingContract, MetaStakingContractVersion
from tools.runners.metastaking_runner import get_metastaking_addresses_from_chain_by_farms
from tools.runners.farm_runner import get_farm_addresses_from_chain
from utils.contract_retrievers import retrieve_proxy_staking_by_address
staking_proxy_contract: MetaStakingContract
metastaking_contracts: list[MetaStakingContract] = []
farm_addresses = get_farm_addresses_from_chain("v2")
staking_proxy_addresses = get_metastaking_addresses_from_chain_by_farms(farm_addresses)
for staking_proxy_address in staking_proxy_addresses:
    staking_proxy_contract = retrieve_proxy_staking_by_address(staking_proxy_address, MetaStakingContractVersion.V2)
    metastaking_contracts.append(staking_proxy_contract)
print(f"Loaded up {len(metastaking_contracts)} contracts")

In [ ]:
for metastaking_contract in metastaking_contracts:
    metastaking_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.STAKING_PROXY_V3_BYTECODE_PATH, [], True)

In [ ]:
for metastaking_contract in metastaking_contracts:
    upgrade_contract_with_check(metastaking_contract, context.deployer_account, context.network_provider.proxy, config.STAKING_PROXY_V3_BYTECODE_PATH, [], True)

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)

for contract in metastaking_contracts:
    tx_hash = contract.set_energy_factory_address(context.deployer_account, context.network_provider.proxy, context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0].address)

Fees collector contract

In [ ]:
from contracts.fees_collector_contract import FeesCollectorContract
fees_collector_contracts: FeesCollectorContract
fees_collector_contracts = context.get_contracts(config.FEES_COLLECTORS)

In [ ]:
for fees_collector_contract in fees_collector_contracts:
    fees_collector_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.FEES_COLLECTOR_BYTECODE_PATH, [], True)

In [ ]:
for fees_collector_contract in fees_collector_contracts:
    upgrade_contract_with_check(fees_collector_contract, context.deployer_account, context.network_provider.proxy, config.FEES_COLLECTOR_BYTECODE_PATH, [], True)

Ownership checkers

In [ ]:
from tools.runners.farm_runner import get_farm_addresses_from_chain
from contracts.farm_contract import FarmContract, FarmContractVersion

farm_addresses = get_farm_addresses_from_chain("v2")
print(farm_addresses)

for address in farm_addresses:
    farm_contract = FarmContract("", "", "", address, FarmContractVersion.V2Boosted)
    print(f'{address} : {farm_contract.get_permissions(config.DEX_OWNER_ADDRESS, context.network_provider.proxy)}')

In [ ]:
from tools.runners.staking_runner import get_staking_addresses_from_chain
from contracts.staking_contract import StakingContract, StakingContractVersion

staking_addresses = get_staking_addresses_from_chain()
print(staking_addresses)

for address in staking_addresses:
    staking_contract = StakingContract("", 0, 0, 0, StakingContractVersion.V3Boosted, "", address)
    print(f'{address} : {staking_contract.get_permissions(config.DEX_OWNER_ADDRESS, context.network_provider.proxy)}')